In [11]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [12]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [13]:
df = df.dropna()
df = shuffle(df, random_state=2)
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [18]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 314us/sample - loss: 1.5340 - mae: 0.9503 - mse: 1.5340 - val_loss: 2.4284 - val_mae: 1.2524 - val_mse: 2.4284
Epoch 2/10
6670/6670 [==============================] - 1s 139us/sample - loss: 1.3508 - mae: 0.9079 - mse: 1.3508 - val_loss: 1.2707 - val_mae: 0.8707 - val_mse: 1.2707
Epoch 3/10
6670/6670 [==============================] - 1s 136us/sample - loss: 1.2801 - mae: 0.8862 - mse: 1.2801 - val_loss: 1.5518 - val_mae: 0.9848 - val_mse: 1.5518
Epoch 4/10
6670/6670 [==============================] - 1s 180us/sample - loss: 1.2325 - mae: 0.8696 - mse: 1.2325 - val_loss: 1.2411 - val_mae: 0.8572 - val_mse: 1.2411
Epoch 5/10
6670/6670 [==============================] - 1s 137us/sample - loss: 1.2021 - mae: 0.8590 - mse: 1.2021 - val_loss: 1.2158 - val_mae: 0.8594 - val_mse: 1.2158
Epoch 6/10
6670/6670 [==============================] - 1s 117us/sample - loss: 1.2093 - mae: 0.8584 - 

In [19]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  6.8297215
Actual val:  6.3823824849

Predicted val:  7.4590216
Actual val:  7.1870697412

Predicted val:  7.166772
Actual val:  7.43839671988

Predicted val:  7.635778
Actual val:  6.6248909731

Predicted val:  7.5835195
Actual val:  5.5005334369

Predicted val:  7.28936
Actual val:  6.686620406459999

Predicted val:  7.9009743
Actual val:  10.306610748499999

Predicted val:  7.041214
Actual val:  4.62529825676

Predicted val:  7.773774
Actual val:  7.7382253962

Predicted val:  7.805752
Actual val:  5.6438339071999994

